# Providers and products

In [1]:
from eodag import EODataAccessGateway
dag = EODataAccessGateway()

## Providers available

The method [available_providers()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.available_providers) returns a list of the pre-configured providers.

In [2]:
available_providers = dag.available_providers()
available_providers

['astraea_eod',
 'aws_eos',
 'cop_ads',
 'cop_cds',
 'cop_dataspace',
 'creodias',
 'creodias_s3',
 'earth_search',
 'earth_search_cog',
 'earth_search_gcs',
 'ecmwf',
 'eumetsat_ds',
 'hydroweb_next',
 'meteoblue',
 'onda',
 'peps',
 'planetary_computer',
 'sara',
 'theia',
 'usgs',
 'usgs_satapi_aws',
 'wekeo']

In [3]:
print(f"eodag has {len(available_providers)} providers already configured.")

eodag has 22 providers already configured.


It can take a product type as an argument and will return the providers known to `eodag` that offer this product.

In [4]:
dag.available_providers("S2_MSI_L1C")

['astraea_eod',
 'aws_eos',
 'cop_dataspace',
 'creodias',
 'creodias_s3',
 'earth_search',
 'earth_search_gcs',
 'onda',
 'peps',
 'sara',
 'usgs',
 'wekeo']

<div class="alert alert-warning">

Note

If a provider is configured to need authentication for search, and has no crendentials set, it will be pruned on EODAG initialization, and will not appear in available providers list.

</div>

## Product types available

The method [list_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types) returns a dictionary that represents `eodag`'s internal product type catalog if used with `fetch_providers=False`. It will fetch providers for new product types and return an extended list if used with `fetch_providers=True` (default behavior).

In [5]:
internal_catalog = dag.list_product_types(fetch_providers=False)
print(f"EODAG has {len(internal_catalog)} product types stored in its internal catalog.")

EODAG has 181 product types stored in its internal catalog.


In [6]:
extended_catalog = dag.list_product_types()
print(f"EODAG has {len(extended_catalog)} product types stored in its extended catalog, after having fetched providers.")

EODAG has 414 product types stored in its extended catalog, after having fetched providers.


When providers are fetched for new product types, `eodag`'s product types configuration is updated in `EODataAccessGateway` instance. Extended product types list is then returned independantly of `fetch_providers` option in [list_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types):

In [7]:
called_again_catalog = dag.list_product_types(fetch_providers=False)
print(f"list_product_types() keeps returning {len(called_again_catalog)} product types.")

list_product_types() keeps returning 414 product types.


In [8]:
internal_catalog[0]

{'ID': 'CAMS_EAC4',
 'abstract': 'EAC4 (ECMWF Atmospheric Composition Reanalysis 4) is the fourth generation ECMWF global reanalysis of atmospheric composition. Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset using a model of the atmosphere based on the laws of physics and chemistry.\nThis principle, called data assimilation, is based on the method used by numerical weather prediction centres and air quality forecasting centres, where every so many hours (12 hours at ECMWF) a previous forecast is combined with newly available observations in an optimal way to produce a new best estimate of the state of the atmosphere, called analysis, from which an updated, improved forecast is issued.\nReanalysis works in the same way to allow for the provision of a dataset spanning back more than a decade.\nReanalysis does not have the constraint of issuing timely forecasts, so there is more time to collect observations, and when 

In [9]:
products_id = [p["ID"] for p in internal_catalog]
products_id

['CAMS_EAC4',
 'CAMS_EAC4_MONTHLY',
 'CAMS_EU_AIR_QUALITY_FORECAST',
 'CAMS_EU_AIR_QUALITY_RE',
 'CAMS_GAC_FORECAST',
 'CAMS_GFE_GFAS',
 'CAMS_GLOBAL_EMISSIONS',
 'CAMS_GREENHOUSE_EGG4',
 'CAMS_GREENHOUSE_EGG4_MONTHLY',
 'CAMS_GREENHOUSE_INVERSION',
 'CAMS_GRF',
 'CAMS_GRF_AUX',
 'CAMS_SOLAR_RADIATION',
 'CBERS4_AWFI_L2',
 'CBERS4_AWFI_L4',
 'CBERS4_MUX_L2',
 'CBERS4_MUX_L4',
 'CBERS4_PAN10M_L2',
 'CBERS4_PAN10M_L4',
 'CBERS4_PAN5M_L2',
 'CBERS4_PAN5M_L4',
 'CLMS_CORINE',
 'CLMS_GLO_DMP_333M',
 'CLMS_GLO_FAPAR_333M',
 'CLMS_GLO_FCOVER_333M',
 'CLMS_GLO_GDMP_333M',
 'CLMS_GLO_LAI_333M',
 'CLMS_GLO_NDVI_1KM_LTS',
 'CLMS_GLO_NDVI_333M',
 'COP_DEM_GLO30_DGED',
 'COP_DEM_GLO30_DTED',
 'COP_DEM_GLO90_DGED',
 'COP_DEM_GLO90_DTED',
 'EEA_DAILY_SSM_1KM',
 'EEA_DAILY_SWI_1KM',
 'EEA_DAILY_VI',
 'EFAS_FORECAST',
 'EFAS_HISTORICAL',
 'EFAS_REFORECAST',
 'EFAS_SEASONAL',
 'EFAS_SEASONAL_REFORECAST',
 'ERA5_LAND',
 'ERA5_LAND_MONTHLY',
 'ERA5_PL',
 'ERA5_PL_MONTHLY',
 'ERA5_SL',
 'ERA5_SL_MONTHLY',


The method can take a provider name as an argument and will return the product types known to `eodag` that are offered by this provider.

In [10]:
peps_products = dag.list_product_types("peps")
[p["ID"] for p in peps_products]

['S1_SAR_GRD', 'S1_SAR_OCN', 'S1_SAR_SLC', 'S2_MSI_L1C']

## Combine these two methods

These two methods can be combined to find which product type is the most common in `eodag`'s catalog among all the providers.

In [11]:
availability_per_product = []
for product in products_id:
    providers = dag.available_providers(product)
    availability_per_product.append((product, len(providers)))
availability_per_product = sorted(availability_per_product, key=lambda x: x[1], reverse=True)
most_common_p_type, nb_providers = availability_per_product[0]
print(f"The most common product type is '{most_common_p_type}' with {nb_providers} providers offering it.")

The most common product type is 'S2_MSI_L1C' with 12 providers offering it.


These can be also used to find out which provider (as configured by `eodag`) offers the hights number of different product types.

In [12]:
availability_per_provider = []
for provider in dag.available_providers():
    provider_products_id = [
        p["ID"]
        for p in dag.list_product_types(provider, fetch_providers=False)
    ]
    availability_per_provider.append(
        (provider, len(provider_products_id))
    )
availability_per_provider = sorted(availability_per_provider, key=lambda x: x[1], reverse=True)
provider, nb_p_types = availability_per_provider[0]
print(f"The provider with the largest number of product types is '{provider}' with {nb_p_types}.")

The provider with the largest number of product types is 'planetary_computer' with 123.


## Product types discovery

EODAG comes with a large list of pre-configured product types. Some others are available from providers catalogs but will not be configured, or are not yet configured in EODAG.

Some providers, like STAC providers, come in EODAG with a configuration describing how to discover these not-already-configured product types.

With the method [discover_product_types()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.discover_product_types) 
or CLI command [eodag discover](../../cli_user_guide.rst) we can obtain a JSON configuration file that will be used as *EODAG 
external product types configuration file*.

In EODAG, the discovered *EODAG external product types configuration file* can be set to:

* a file automatically built from github actions and stored in [eodag/resources/ext_product_types.json](https://raw.githubusercontent.com/CS-SI/eodag/develop/eodag/resources/ext_product_types.json) (default settings)
* a custom remote or local file by setting its path in `EODAG_EXT_PRODUCT_TYPES_CFG_FILE` environment variable (if the file is not readable, only user-modified providers will be fetched).

Then, when listing product types using [list_product_types(fetch_providers=True)](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.list_product_types), EODAG will first read the content of the *EODAG external product types configuration file* using [fetch_product_types_list()](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway.fetch_product_types_list) 
then update [EODataAccessGateway](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway) instance product types configuration, if needed.

The obtained product types list will contain both pre-configured and discovered product types.

![Fetch product types schema](../../_static/eodag_fetch_product_types.png "Fetch product types schema")